## Global Database of Humanitarian Organizations

I got this csv by searching all orgs with operations in the hotspot countries + Sahel region, and all operations. I figured we can filter down the categories later

In [1]:
pip install skimpy --user

     |████████████████████████████████| 12.4 MB 38.1 MB/s eta 0:00:01
     |████████████████████████████████| 239 kB 102.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 93.9 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 13.0 MB/s  eta 0:00:01
     |████████████████████████████████| 341 kB 102.7 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 92.5 MB/s eta 0:00:01


  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from skimpy import clean_columns
import time
import requests
pd.set_option('display.max_columns', 500)


In [3]:
gdho = pd.read_csv("gdho.csv")

In [4]:
gdho = clean_columns(gdho)

In [5]:
gdho.head()

,id,year,name,abbreviated_name,type,international_or_national,website,hq_location,year_founded,year_closed,countries_of_operation_count,sector,religious_or_secular,religion,red_cross_code_of_conduct_signatory,chs_member,inter_action_member,icva_member,staff,staff_imputed,natl,natl_imputed,intl,intl_imputed,%_intl,ope_approx_usd,ope_imputed,ope_staff,ope_inflation_adjusted,ope_original_currency,hum_exp_approx_usd,hum_exp_imputed,hum_exp_inflation_adjusted,afghanistan,albania,algeria,american_samoa,andorra,angola,anguilla,antarctica,antigua_and_barbuda,argentina,armenia,aruba,australia,austria,azerbaijan,bahamas,bahrain,bangladesh,barbados,belarus,belgium,belize,benin,bermuda,bhutan,bolivia,bosnia_and_herzegovina,botswana,bouvet_island,brazil,british_indian_ocean_territory,brunei_darussalam,bulgaria,burkina_faso,burundi,cambodia,cameroon,canada,cape_verde,cayman_islands,central_african_republic,chad,chile,china,christmas_island,cocos_keeling_islands,colombia,comoros,congo,cook_islands,costa_rica,cote_d_ivoire,croatia,cuba,cyprus,czech_republic,denmark,djibouti,dominica,dominican_republic,dr_congo,ecuador,egypt,el_salvador,equatorial_guinea,eritrea,estonia,ethiopia,falkland_islands_malvinas,faroe_islands,fiji,finland,france,french_guiana,french_polynesia,french_southern_territories,gabon,gambia,georgia,germany,ghana,gibraltar,greece,greenland,grenada,guadeloupe,guam,guatemala,guinea,guinea_bissau,guyana,haiti,heard_island_and_mcdonald_islands,holy_see_vatican_city_state,honduras,hong_kong,hungary,iceland,india,indonesia,iran_islamic_republic_of,iraq,ireland,israel,italy,jamaica,japan,jordan,kazakhstan,kenya,kiribati,korea_democratic_peoples_republic_of,korea_republic_of,kuwait,kyrgyzstan,lao_peoples_democratic_republic,latvia,lebanon,lesotho,liberia,libyan_arab_jamahiriya,liechtenstein,lithuania,luxembourg,macao,macedonia_the_former_yugoslav_republic_of,madagascar,malawi,malaysia,maldives,mali,malta,marshall_islands,martinique,mauritania,mauritius,mayotte,mexico,micronesia_federated_states_of,moldova_republic_of,monaco,mongolia,montserrat,morocco,mozambique,myanmar,namibia,nauru,nepal,netherlands,netherlands_antilles,new_caledonia,new_zealand,nicaragua,niger,nigeria,niue,norfolk_island,northern_mariana_islands,norway,occupied_palestinian_territories,oman,pakistan,palau,panama,papua_new_guinea,paraguay,peru,philippines,pitcairn,poland,portugal,puerto_rico,qatar,reunion,romania,russian_federation,rwanda,saint_helena,saint_kitts_and_nevis,saint_lucia,saint_pierre_and_miquelon,saint_vincent_and_the_grenadines,samoa,san_marino,sao_tome_and_principe,saudi_arabia,senegal,serbia_and_montenegro,seychelles,sierra_leone,singapore,slovakia,slovenia,solomon_islands,somalia,south_africa,south_georgia_and_the_south_sandwich_islands,south_sudan,spain,sri_lanka,sudan,suriname,svalbard_and_jan_mayen,swaziland,sweden,switzerland,syrian_arab_republic,taiwan,tajikistan,tanzania,thailand,timor_leste,togo,tokelau,tonga,trinidad_and_tobago,tunisia,turkey,turkmenistan,turks_and_caicos_islands,tuvalu,uganda,ukraine,united_arab_emirates,united_kingdom,united_states,united_states_minor_outlying_islands,uruguay,uzbekistan,vanuatu,venezuela,vietnam,virgin_islands_british,virgin_islands_u_s,wallis_and_futuna,western_sahara,yemen,zambia,zimbabwe
0,22,2020,Action Africa Help-International,AAH-I,INGO,International,http://www.actionafricahelp.org,Kenya,1987.0,NaN,4,NaN,Secular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$14,658,708.00",A,"$17,682.40","$16,578,200.00",NaN,"$12,833,628.00",A,"$14,514,134.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,N

In [6]:
gdho["sector"].unique()

array([nan, 'Health', 'Education', 'Food security/agriculture',
       'Children/youth', 'Disabilities', 'Shelter', 'Refugees',
       'Landmines', 'Livelihoods', 'Water/sanitation', 'Women',
       'Coordination', 'Environment', 'Nutrition ',
       'Logistics/Communications'], dtype=object)

In [7]:
country_list = ['Angola',
 'Burkina%20Faso',
 'Central%20African%20Republic',
 'Chad',
 "cote_d_ivoire",
 'dr_congo',
 'Djibouti',
 'Eritrea',
 'Ethiopia',
 'Guinea',
 'Liberia',
 'Mali',
 'Mauritania',
 'Niger',
 'Nigeria',
 'Senegal',
 'Sierra%20Leone',
 'Somalia',
 'South%20Sudan',
 'Sudan',
 'Zambia']
country_list = [x.lower() for x in country_list]
country_list = [x.replace("%20", "_") for x in country_list]

In [8]:
select_columns = ["id", "year", "name", "abbreviated_name", "type", "international_or_national", "website", "hq_location", "year_founded", "year_closed", "sector", "countries_of_operation_count"] + country_list

In [9]:
select_columns

['id',
 'year',
 'name',
 'abbreviated_name',
 'type',
 'international_or_national',
 'website',
 'hq_location',
 'year_founded',
 'year_closed',
 'sector',
 'countries_of_operation_count',
 'angola',
 'burkina_faso',
 'central_african_republic',
 'chad',
 'cote_d_ivoire',
 'dr_congo',
 'djibouti',
 'eritrea',
 'ethiopia',
 'guinea',
 'liberia',
 'mali',
 'mauritania',
 'niger',
 'nigeria',
 'senegal',
 'sierra_leone',
 'somalia',
 'south_sudan',
 'sudan',
 'zambia']

In [10]:
df = gdho[select_columns]

In [11]:
df= df[df["sector"] != "Logistics/Communications"]

In [12]:
df = df.drop_duplicates()

In [13]:
df.shape

(519, 33)

In [14]:
df['countries'] = df.apply(lambda row: row.index[row == 1].tolist(), axis=1)

In [15]:
df.head()

,id,year,name,abbreviated_name,type,international_or_national,website,hq_location,year_founded,year_closed,sector,countries_of_operation_count,angola,burkina_faso,central_african_republic,chad,cote_d_ivoire,dr_congo,djibouti,eritrea,ethiopia,guinea,liberia,mali,mauritania,niger,nigeria,senegal,sierra_leone,somalia,south_sudan,sudan,zambia,countries
0,22,2020,Action Africa Help-International,AAH-I,INGO,International,http://www.actionafricahelp.org,Kenya,1987.0,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,"[somalia, zambia]"
2,23,2020,Action Contre la Faim International (ACF/ACH/AAH),ACF,INGO,International,http://www.actioncontrelafaim.org,France,1979.0,NaN,NaN,44,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,"[angola, burkina_faso, central_african_republi..."
14,32,2021,ActionAid International,NaN,INGO,International,http://www.actionaid.org,South Africa,1972.0,NaN,NaN,45,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,"[dr_congo, ethiopia, liberia, nigeria, senegal..."
21,40,2021,Adventist Development and Relief Agency,ADRA,INGO,International,http://www.adra.org/,United States,1956.0,NaN,NaN,74,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,"[burkina_faso, chad, cote_d_ivoire, dr_congo, ..."
33,51,2021,Africa Humanitarian Action,AHA,INGO,International,http://www.africahumanitarian.org,Ethiopia,1994.0,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,"[ethiopia, south_sudan, sudan]"


In [16]:
search_queries = []
for index, row in df.iterrows():
    for country in row["countries"]:
        query = row["name"] + " " + country
        search_queries.append(query)
        
search_queries = [x.replace("_", " ") for x in search_queries]

In [17]:
len(search_queries)

2059

In [18]:
search_queries

['Action Africa Help-International somalia',
 'Action Africa Help-International zambia',
 'Action Contre la Faim International (ACF/ACH/AAH) angola',
 'Action Contre la Faim International (ACF/ACH/AAH) burkina faso',
 'Action Contre la Faim International (ACF/ACH/AAH) central african republic',
 'Action Contre la Faim International (ACF/ACH/AAH) chad',
 'Action Contre la Faim International (ACF/ACH/AAH) cote d ivoire',
 'Action Contre la Faim International (ACF/ACH/AAH) dr congo',
 'Action Contre la Faim International (ACF/ACH/AAH) mali',
 'Action Contre la Faim International (ACF/ACH/AAH) mauritania',
 'Action Contre la Faim International (ACF/ACH/AAH) nigeria',
 'Action Contre la Faim International (ACF/ACH/AAH) somalia',
 'Action Contre la Faim International (ACF/ACH/AAH) south sudan',
 'Action Contre la Faim International (ACF/ACH/AAH) sudan',
 'ActionAid International dr congo',
 'ActionAid International ethiopia',
 'ActionAid International liberia',
 'ActionAid International nige

## Get coordinates from Google maps

In [23]:
def search_google_maps(query):
    base_url = "https://www.google.com/maps/search/"
    search_url = f"{base_url}?q={query}"
    
    response = requests.get(search_url)
    return response.text

In [21]:
def get_coordinates(query):
    location = search_google_maps(query)
    index_key = "APP_INITIALIZATION_STATE=[[["
    index = location.find(index_key)
    if index != -1:
        # Extract the 55 characters after the found index
        extracted_text = location[index + len(index_key): index + len(index_key) + 55]
        split_text = extracted_text.split(",")
    else:
        split_text = "Location not found"
    return split_text

### read in existing org list

In [19]:
orgs_df = pd.read_csv("gdho_locations.csv")
orgs_df = orgs_df[orgs_df["lat"] != "c"]
orgs_df.shape

(200, 3)

In [24]:
org_locations = []
for org in search_queries[306:400]:
    location = get_coordinates(org)
    org_loc = {
        "name": org,
        "lat": location[1:][1].replace("]", ""),
        "long": location[1:][0]
    }
    org_locations.append(org_loc)
    
    time.sleep(5)

In [25]:
temp_df = pd.DataFrame(org_locations)

In [26]:
temp_df.shape

(150, 3)

In [27]:
orgs_df.shape

(200, 3)

In [30]:
temp_df.head()

,name,lat,long
0,Concern Worldwide chad,51.4670711796284,-0.1833518842315117
1,Concern Worldwide dr congo,-1.68636139877031,29.231898014754332
2,Concern Worldwide ethiopia,11.25142196143362,36.911431867032434
3,Concern Worldwide liberia,6.29403429544468,-10.789771585230511
4,Concern Worldwide nigeria,38.900355779570205,-77.0479645846496


In [32]:
temp_df = temp_df[temp_df["lat"] != "c"]

In [33]:
orgs_df = pd.concat([temp_df, orgs_df])

In [34]:
orgs_df.shape

(306, 3)

In [35]:
orgs_df.to_csv("gdho_locations.csv", index = False)

In [76]:
orgs_df.shape

(250, 3)